In [236]:
HOST = 'http://localhost:8123'
import requests
import pandas as pd
import StringIO
import datetime
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 150)

def get_clickhouse_data(query, host = HOST, connection_timeout = 1500):
    query = query 
    r = requests.post(host, params = {'query': query}, timeout = connection_timeout)
    if r.status_code == 200:
        return r.text
    else:
        raise ValueError, r.text
        
def get_clickhouse_df(query, host = HOST, connection_timeout = 1500):
    data = get_clickhouse_data(query, host, connection_timeout) 
    df = pd.read_csv(StringIO.StringIO(data), sep = '\t')
    return df

In [237]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
import requests
import StringIO
import pandas as pd

print __version__ # need 1.9.0 or greater

init_notebook_mode(connected = True)

def plotly_df(df, title = ''):
    data = []
    
    for column in df.columns:
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            name = column
        )
        data.append(trace)
    
    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    
    # plotly.offline.plot(fig, filename=filename, show_link = False)
    
    iplot(fig, show_link = False)

1.12.9


In [238]:
def highlight_vals(val):
    if (val is None) or (val == ''):
        return ''
    p = 0.5
    if val > 90:
        return 'background-color: rgba(229, 0, 20, %f)' % p
    if val > 80:
        return 'background-color: rgba(231, 25, 43, %f)' % p
    if val > 70:
        return 'background-color: rgba(234, 51, 67, %f)' % p
    if val > 60:
        return 'background-color: rgba(236, 76, 90, %f)' % p
    if val > 50:
        return 'background-color: rgba(239, 102, 114, %f)' % p
    if val > 40:
        return 'background-color: rgba(242, 137, 127, %f)' % p
    if val > 30:
        return 'background-color: rgba(244, 153, 161, %f)' % p
    if val > 20:
        return 'background-color: rgba(247, 178, 184, %f)' % p
    if val > 10:
        return 'background-color: rgba(249, 204, 208, %f)' % p
    return 'background-color: rgba(252, 229, 231, %f)' % p

In [239]:
start_date = datetime.datetime(2016, 11, 7)
end_date = datetime.datetime(2016, 11, 25)

In [240]:
print get_clickhouse_data("DESCRIBE visits_all")

Bounce	UInt8		
Browser	String		
ClientID	UInt64		
CounterID	UInt32		
Date	Date		
DateTime	DateTime		
DeviceCategory	String		
GoalsID	Array(UInt32)		
IsNewUser	UInt8		
LastAdvEngine	String		
LastSearchEngineRoot	String		
LastSocialNetwork	String		
LastTrafficSource	String		
MobilePhone	String		
OperatingSystemRoot	String		
PageViews	Int32		
Params	Array(String)		
RegionCity	String		
RegionCountry	String		
StartURL	String		
UTMMedium	String		
VisitDuration	UInt32		
VisitID	UInt64		



In [244]:
q = 'DROP TABLE retention_users'
get_clickhouse_data(q)

u''

In [245]:
q = '''
    CREATE TABLE retention_users ENGINE = Log AS
        SELECT DISTINCT ClientID as client_id, toMonday(Date) as date
        FROM visits_all
        WHERE (client_id != 0)
'''.format(
    start_date = start_date.strftime('%Y-%m-%d'),
    end_date = end_date.strftime('%Y-%m-%d')
)

get_clickhouse_data(q)


u''

In [246]:
q = '''SELECT count(), uniq(client_id) FROM retention_users FORMAT TabSeparatedWithNames'''
get_clickhouse_df(q)

,count(),uniq(client_id)
0,971515,851035


In [247]:
q = '''
SELECT 
    client_id,
    min_date, 
    max_date,
    date
FROM
    (
        SELECT
            client_id,
            min(date) as min_date,
            max(date) as max_date
        FROM retention_users
        GROUP BY client_id
        HAVING (min_date <= '{end_date}') AND (min_date >= '{start_date}')
    )
    ALL INNER JOIN
    (
        SELECT 
            client_id,
            date
        FROM retention_users
    ) 
    USING client_id
LIMIT 10
FORMAT TabSeparatedWithNames
'''.format(
    start_date = start_date.strftime('%Y-%m-%d'),
    end_date = end_date.strftime('%Y-%m-%d')
)

get_clickhouse_df(q)

,client_id,min_date,max_date,date
0,1463659610347692840,2016-11-21,2016-11-28,2016-11-21
1,1463659610347692840,2016-11-21,2016-11-28,2016-11-28
2,1479994646578665295,2016-11-21,2016-11-21,2016-11-21
3,1476856735518320455,2016-11-21,2016-11-21,2016-11-21
4,1478245642707077152,2016-11-14,2016-11-14,2016-11-14
5,1472586969961543035,2016-11-07,2016-11-07,2016-11-07
6,1476355428687274143,2016-11-14,2016-11-14,2016-11-14
7,147257294157910964,2016-11-14,2016-11-14,2016-11-14
8,1478460311786345588,2016-11-07,2016-11-07,2016-11-07
9,1471451103864029657,2016-11-07,2016-11-07,2016-11-07


## Простой retention

In [249]:
q = '''
SELECT 
    uniq(client_id) as clients,
    min_date, 
    (date - min_date)/7 as week_num
FROM
    (
        SELECT
            client_id,
            min(date) as min_date,
            max(date) as max_date
        FROM retention_users
        GROUP BY client_id
        HAVING (min_date <= '{end_date}') AND (min_date >= '{start_date}')
    )
    ALL INNER JOIN
    (
        SELECT 
            client_id,
            date
        FROM retention_users
    ) 
    USING client_id
GROUP BY
    week_num,
    min_date
FORMAT TabSeparatedWithNames
'''.format(
    start_date = start_date.strftime('%Y-%m-%d'),
    end_date = end_date.strftime('%Y-%m-%d')
)

raw_ret_df = get_clickhouse_df(q)

In [250]:
ret_df = raw_ret_df.pivot_table(index = 'min_date', values = 'clients', columns = 'week_num').fillna(0).T

In [251]:
ret_df_norm = ret_df.apply(lambda x: 100*x/ret_df.loc[0], axis = 1).applymap(lambda x: x if x!=0 else None)

In [252]:
plotly_df(ret_df_norm)

In [254]:
ret_df_norm.T.fillna('').style.applymap(highlight_vals)

## Rolling retention

In [255]:
q = '''
SELECT
    uniq(client_id) as clients,
    min_date,
    week_num
FROM
    (SELECT 
        client_id,
        min_date, 
        arrayJoin(range(toUInt64((max_date - min_date)/7) + 1)) as week_num
    FROM
        (
            SELECT
                client_id,
                min(date) as min_date,
                max(date) as max_date
            FROM retention_users
            GROUP BY client_id
            HAVING (min_date <= '{end_date}') AND (min_date >= '{start_date}')
        ))
GROUP BY
    min_date,
    week_num
FORMAT TabSeparatedWithNames
'''.format(
    start_date = start_date.strftime('%Y-%m-%d'),
    end_date = end_date.strftime('%Y-%m-%d')
)

raw_roll_ret_df = get_clickhouse_df(q)

In [256]:
roll_ret_df = raw_roll_ret_df.pivot_table(index = 'min_date', 
                                          values = 'clients', 
                                          columns = 'week_num').fillna(0).T

In [257]:
roll_ret_df_norm = roll_ret_df.apply(lambda x: 100*x/roll_ret_df.loc[0], axis = 1).applymap(lambda x: x if x!=0 else None)

In [258]:
plotly_df(roll_ret_df_norm)

In [259]:
roll_ret_df_norm.T.fillna('').style.applymap(highlight_vals)